In [1]:
from supertypes import *
import math
Sqrt = Fn(math.sqrt, name="Sqrt")
x, y, z = Meta('x'), Meta('y'), Meta('z')

In [3]:
Type([[7, 8, 9], {'a': 1, 'b': 2}])[0][0](1.4)

1

In [1]:
from gpt import *
rx = '[a-zA-Z\-\(\),\.;: ]+'
enc = tiktoken.get_encoding("cl100k_base")
i = 0
tokenDict = Dict()
while i < 100256: 
	tokenDict[i] = enc.decode([i])
	i += 1
cleanDict = tokenDict.filter(lambda k, v: re.fullmatch(rx, v) and len(v) > 3)

Checking available models against listed models
 -- 4 models available but unlisted (dall-e-2, dall-e-3, gpt-4-vision-preview, whisper-1)
 -- 3 models listed but unavailable (gpt-4-32k, gpt-4-32k-0314, gpt-4-32k-0613)


In [209]:
def convert(fp): 
	f = open(fp, "r").read()
	L = enc.encode(f)
	D, E = {}, {}
	for k in cleanDict.keys(): 
		if (c := L.count(k)) > 0 and k not in D.keys(): 
			D[k] = c
			E[cleanDict[k]] = c
	D1 = sorted(D.items(), key=lambda x: x[1], reverse=True)
	D = {k: v for k, v in D1}
	return D

def add(*args):
	if len(args) > 2: 
		return add(args[0], add(*args[1:]))
	Da, Db = args[0], args[1]
	D = {}
	a, b = sum(Da.values()), sum(Db.values())
	for k in set(list(Da.keys()) + list(Db.keys())): 
		D[k] = Da.get(k, 0) / a + Db.get(k, 0) / b
	return D

def diff(Da, Db): 
	D = {}
	a, b = sum(Da.values()), sum(Db.values())
	for k in set(list(Da.keys()) + list(Db.keys())): 
		D[k] = Da.get(k, 0) / a - Db.get(k, 0) / b
	return D

def truncate(D, w, minlen = 3, onlypos = True): 
	D = {k: v for k, v in D.items() if len(enc.decode([k]))-enc.decode([k]).count(' ') >= minlen}
	if onlypos: 
		D = {k: v for k, v in D.items() if v > 0}
	currentWeight = sum([abs(x) for x in D.values()])
	D1 = sorted(D.items(), key=lambda x: abs(x[1]), reverse=True)
	logit_bias = {k: w * v / currentWeight for k, v in D1[20:320]}
	return logit_bias

def test(prompt, logit_bias): 
	print(completion(prompt, {"logit_bias": logit_bias, "temperature": 0.1}))
	print('------'*10)
	print(completion(prompt, {"temperature": 0.1}))

def testChat(prompt, logit_bias): 
	Chatbot = GPT(model='gpt-4')
	chat = lambda content, role="user", settings={}: Chatbot.run(content, settings, role=role)
	print(chat(prompt, role="user", settings={"logit_bias": logit_bias, "temperature": 0.1}))

show = lambda x: sorted([(enc.decode([k]), v) for k, v in x.items()], key=lambda x: abs(x[1]), reverse=True)

In [199]:
wiki = convert('./Wikipedia.html') # 11.3s
critique1 = convert("../html/CPR.html") # 2m 30.8s
critique2 = convert("../html/critique2.html") # 39.9s
critique3 = convert("../html/critique3.html") # 1m 41.1s
thoughts = convert('../html/thoughts.html') # 1m 1.2s
worldspace = convert("../html/worldspace.html") # 39.2s
ecm = convert("../html/ecm.html") # 13.4s
# total = 7m 5s
kant = add(critique1, critique2, critique3)
mar = add(thoughts, worldspace, formiri, ecm)

In [203]:
log_probs = truncate(diff(diff(mar, kant), wiki), 2000)